In [8]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/ielyaascloete/auto/07p/python")
from auto import *
import scipy.optimize as op

In [9]:
Kf=10; Kb=0.4; Kp=0.2; Kc=0.2; Kh=0.08; tauP=1;
Vs=0.9; Kbar=1.957e-5; Ks=0.2;
gamma=5.5; h=0.001;
Kplc=0.1; R_act=0.51;
a0=0.0027; a1=0.015; Kce=14;
Vpm=0.11; Kpm=0.3; delta=2.5;

def f(x):
    
    m = x[0]**4/(x[0]**4+Kc**4)
    h_inf = Kh**4/(Kh**4+x[0]**4)
    b = x[2]**2/(Kp**2+x[2]**2)
    a = 1-b
    beta = b*m*h
    alpha = a*(1-m*h_inf)
    Po = beta/(beta+Kb*(beta+alpha))
    PLC = R_act*x[0]**2/(Kplc**2+x[0]**2)
    ce = gamma*(x[1]-x[0])
    Jserca = Vs*(x[0]**2-Kbar*ce**2)/(x[0]**2+Ks**2)
    Jipr = Kf*Po*(ce-x[0])
    Jin = a0+a1*(Kce**4/(ce**4+Kce**4))
    Jpm = Vpm*x[0]**2/(x[0]**2+Kpm**2)
    
    return np.array([Jipr-Jserca,delta*(Jin-Jpm),tauP*(PLC-x[2])])

sol = op.root(f,x0=np.array([0.07800382606144,3.283961636746565,0.01929264644940769]))
sol.x

array([0.079696  , 3.2129949 , 0.19810111])

In [10]:
sol

    fjac: array([[ 0.67670319,  0.09463179, -0.73014904],
       [-0.72461725,  0.26123783, -0.6377183 ],
       [-0.13039413, -0.9606246 , -0.2453523 ]])
     fun: array([ 4.08443158e-11, -3.96103072e-12, -1.97455940e-12])
 message: 'The solution converged.'
    nfev: 10
     qtf: array([-5.91986102e-10,  6.40650131e-10,  1.92631767e-11])
       r: array([-4.18482694,  0.0469641 ,  0.80168612, -0.05409123,  0.56603699,
        0.23126967])
  status: 1
 success: True
       x: array([0.079696  , 3.2129949 , 0.19810111])

In [11]:
f(sol.x)

array([ 4.08443158e-11, -3.96103072e-12, -1.97455940e-12])

In [12]:
import sys
print(sys.version)

2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [13]:
cl

<function AUTOCommands.cl>

In [14]:
Eq = run(e='fast_slow_h',c='fast_slow_h.eq')

gfortran -fopenmp -O -c fast_slow_h.f90 -o fast_slow_h.o
gfortran -fopenmp -O fast_slow_h.o -o fast_slow_h.exe /Users/ielyaascloete/auto/07p/lib/*.o


Undefined symbols for architecture x86_64:
  "_getp_", referenced from:
      _pvls_ in fast_slow_h.o
  "_main", referenced from:
     implicit entry/start for main executable
ld: symbol(s) not found for architecture x86_64
collect2: error: ld returned 1 exit status


AUTORuntimeError: Error running AUTO

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'SNP' Saddle node of periodic orbits
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()

    """Read the file from the beginning of the interesting part (depends on its type)"""
    if solution_type == 'EQ':
        content_csv = [[el for el in content[14].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'PR':
        content_csv = [[el for el in content[16].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'HB':
        content_csv = [[el for el in content[17].split(' ') if len(el) > 0 and el != '\n']]
    
    elif solution_type == 'SNP':
        content_csv = [[el for el in content[18].split(' ') if len(el) > 0 and el != '\n']]
        
    elif solution_type == 'SN':
        content_csv = [[el for el in content[17].split(' ') if len(el) > 0 and el != '\n']]
    else:
        print('Solution type not supported')
        return;
    
        
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
    print(column_names)
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [ ]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
trace_eq = Scatter(x=df_eq['ct'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
data = [trace_eq]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='[Ct]'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_eqs = []

for h in np.linspace(0.0031,0.41,500):
    initial = run(e='hep_fixed_h',c='hep_fixed_h.eq',RL1 = h,ICP = [2])
    Eq = run(initial('EP2'),c='hep_fixed_h.eq',ICP =  [1,2,5,6,7,8,80])
   
    df_eqs.append(auto_to_csv(Eq,'Eq_h'+str(h),'EQ'))
    